<a href="https://colab.research.google.com/github/shambhavithakur/marketing-subscription-predictor/blob/main/marketing_subs_model_creation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting whether or not a customer will purchase a marketing subscription: Model creation with plain Scikit-Learn [3]

## Introduction

In this notebook, we will train models using selected features from the prepared dataset. To find out how this dataset was prepared, refer to the [exploratory data analysis notebook](https://colab.research.google.com/github/shambhavithakur/marketing-subscription-predictor/blob/main/marketing_subs_eda.ipynb).

As the target variable in the dataset is imbalanced, we will train a few tree-based models, Gaussian Naive Bayes (GaussianNB), Gradient Boosting Classifier (GBC), Ada Boost Classifier (ABC), Decision Tree Classifier (DTC), Random Forest Classifier (RFC), and XGBoost, on the data. Compared to other types of models, tree-based models generate more accurate predictions on imbalanced&nbsp;data.

## Loading the training data set

In [36]:
#@title
import pandas as pd

pd.set_option('precision', 3)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', '{:.3f}'.format)

In [1]:
# Using pandas to load the dataset

base_path = '/content/drive/MyDrive/ml/20201015-marketing-subs/'

train = pd.read_feather(base_path + 'train')
test = pd.read_feather(base_path + 'test')

In [2]:
# Creating copies of the train and test data sets with selected features

selected_features = ['age', 'job', 'marital', 'education', 'default', 'housing',\
                     'loan', 'contact', 'day_of_week', 'day_of_week_cos',\
                     'campaign', 'previous', 'poutcome', 'cons_conf_idx',\
                     'euribor3m', 'nr_employed', 'bought']

train_v2 = train[selected_features].copy()
test_v2 = test[selected_features].copy()

In [3]:
train_v2.shape, test_v2.shape

((35511, 17), (3946, 17))

In [4]:
train_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35511 entries, 0 to 35510
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   age              35507 non-null  float64 
 1   job              35511 non-null  category
 2   marital          35511 non-null  category
 3   education        35511 non-null  category
 4   default          35511 non-null  category
 5   housing          35511 non-null  category
 6   loan             35511 non-null  category
 7   contact          35506 non-null  category
 8   day_of_week      35511 non-null  category
 9   day_of_week_cos  35511 non-null  category
 10  campaign         35511 non-null  int64   
 11  previous         35511 non-null  category
 12  poutcome         35511 non-null  category
 13  cons_conf_idx    35511 non-null  float64 
 14  euribor3m        35511 non-null  float64 
 15  nr_employed      35511 non-null  category
 16  bought           35511 non-null  categor

## Subsetting the training data into training and validation subsets

In [5]:
from sklearn.model_selection import train_test_split

seed = 42

X = train_v2.drop('bought', axis=1)
y = train_v2.copy().bought

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

## Separating numerical and categorical columns

In the subsequent steps, we intend to use Scikit-Learn pipelines to impute missing data into the datasets. Scikit-Learn has different imputers for numerical and categorical data. Each of these imputers requires a list containing the names of the columns that it must impute. Therefore, it is imperative that we create separate lists of numerical and categorical columns.


In [6]:
import numpy as np

categorical_cols = list(X.select_dtypes(include=['object', 'category']).columns)
numerical_cols = list(X.select_dtypes(include=[np.number]).columns)

## Training GaussianNB, GBC, ABC, DTC, and RFC models

In [7]:
# Importing relevant modules

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import KBinsDiscretizer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, MaxAbsScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [8]:
# Creating a preprocessor to impute missing values
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ("ohe", OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# Setting up recursive feature elimination (RFE) to make sure features are selected 
# by recursively considering smaller and smaller sets of features
rfe = RFE(estimator=LogisticRegression(solver='lbfgs', class_weight='balanced', max_iter=1000),\
          n_features_to_select=7)

In [9]:
# Writing a function to determine accuracy, F1 score, and 
# area under the ROC curve (AUROC)

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def get_metrics(model, model_name, X=X, y=y, test=test_v2, target='bought'):
    # Splitting test data
    test_X = test.drop(target, axis=1)
    test_y = test.copy()[target]
    
    # Fitting model on entire training data
    model.fit(X, y)

    # Obtaining prediction on new data
    y_pred = model.predict(test_X)

    print(f'Data about the {model_name} model:')

    # Finding out whether the model is predicting both classes
    print(f'Classes predicted by the model: {np.unique(y_pred)}')

    # Calculating accuracy score
    print(f'Accuracy score: {accuracy_score(test_y, y_pred):.3f}')

    # Calculating F1 score
    print(f'F1 score: {f1_score(test_y, y_pred):.3f}')

    # Determining AUROC
    y_prob = model.predict_proba(test_X)
    y_prob = [p[1] for p in y_prob]
    print(f'AUROC of the model: {roc_auc_score(test_y, y_prob):.3f}')

In [10]:
# Training a GaussianNB model
from sklearn.base import TransformerMixin

class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

pipe_gnb = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feat_selector', rfe),
                           ('to_dense', DenseTransformer()),
                           ('gnb', GaussianNB())
                          ])

mod_gnb = pipe_gnb.fit(X_train, y_train)

# Evaluating the GaussianNB model
get_metrics(mod_gnb, 'GaussianNB')

Data about the GaussianNB model:
Classes predicted by the model: [0 1]
Accuracy score: 0.878
F1 score: 0.395
AUROC of the model: 0.745


In [11]:
# Training an ABC model
pipe_abc = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feat_selector', rfe),
                           ('abc', AdaBoostClassifier(random_state=seed,\
                                                      n_estimators=500))
                          ])

mod_abc = pipe_abc.fit(X_train, y_train)

# Evaluating the ABC model
get_metrics(mod_abc, 'ABC')

Data about the ABC model:
Classes predicted by the model: [0 1]
Accuracy score: 0.885
F1 score: 0.295
AUROC of the model: 0.746


In [ ]:
# Training a DTC model
pipe_dtc = Pipeline(steps=[('preprocessor', preprocessor),
                           ('maxabs', MaxAbsScaler()),
                           ('feat_selector', rfe),
                           ('dtc', DecisionTreeClassifier(max_features='log2',\
                                                          random_state=seed))
                           ])

mod_dtc = pipe_dtc.fit(X_train, y_train)

# Evaluating the DTC model
get_metrics(mod_dtc, 'DTC')

In [ ]:
# Training an RFC model
pipe_rfc = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feat_selector', rfe),
                           ('rfc', RandomForestClassifier(n_estimators=500,\
                                                          max_features=0.9,\
                                                          random_state=seed))
                           ])

mod_rfc = pipe_rfc.fit(X_train, y_train)

# Evaluating the RFC model
get_metrics(mod_rfc, 'RFC')

In [ ]:
# Training a GBC model
pipe_gbc = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feat_selector', rfe),
                           ('gbc', GradientBoostingClassifier(random_state=seed,\
                                                              max_depth=5,\
                                                              n_estimators=500,\
                                                              warm_start=True,\
                                                              n_iter_no_change=10))
                          ])

mod_gbc = pipe_gbc.fit(X_train, y_train)

# Evaluating the GBC model
get_metrics(mod_gbc, 'GBC')

In [14]:
# Training an XGBoost classifier using random search

from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
])

pipe_grid = Pipeline([('preprocessor', preprocessor), ('feat_selector', rfe)])

xgb = XGBClassifier(tree_method='gpu_hist', max_bin=16, verbosity=0)

pipe_xgb = Pipeline([('xgb', xgb)])

prams={
    'xgb__learning_rate': [0.01,0.03,0.05,0.1,0.15,0.2],
    'xgb__n_estimators': [100,200,500,1000,2000],
    'xgb__max_depth': [3,5,10],
    'xgb__colsample_bytree': [0.1,0.3,0.5,1],
    'xgb__subsample': [0.1,0.3,0.5,1]
}

searcher = RandomizedSearchCV(pipe_xgb, param_distributions=prams, verbose=10, n_iter=20, \
                              random_state=seed, cv=10, scoring='roc_auc', n_jobs=-1,\
                              error_score=0, refit=True)

random_search_xgb = make_pipeline(pipe_grid, searcher)
random_search_xgb.fit(X_train, y_train) 

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  6

Pipeline(memory=None,
         steps=[('pipeline',
                 Pipeline(memory=None,
                          steps=[('preprocessor',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='drop',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('num',
                                                                   Pipeline(memory=None,
                                                                            steps=[('imputer',
                                                                                    SimpleImputer(add_indicator=False,
                                                                                                  copy=True,
                                                                                       

In [15]:
# Evaluating the XGBoost model
get_metrics(random_search_xgb, 'XGBoost')

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   59.8s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  6

Data about the XGBoost model:
Classes predicted by the model: [0 1]
Accuracy score: 0.885
F1 score: 0.295
AUROC of the model: 0.746


## Saving models

In [16]:
from joblib import dump
dump(mod_gnb, 'gaussiangb_plain_sklearn.joblib')
dump(random_search_xgb, 'random_search_xgb_plain_sklearn.joblib')

['random_search_xgb_plain_sklearn.joblib']

## Loading the saved GaussianGB model and making a prediction

In [37]:
# Defining a function to get prediction from a loaded model

def get_prediction_loaded_model(model, model_name, test, target='bought'):
    # Splitting test data
    test_X = test.drop(target, axis=1)

    # Obtaining prediction on new data
    y_pred = model.predict(test_X)

    print(f'{model_name} prediction: {y_pred}')

    print('\nThe prediction is for the following data:')
    print(test.head())


In [42]:
from joblib import load

loaded_gaussiangb_plain_sklearn = load('gaussiangb_plain_sklearn.joblib')

get_prediction_loaded_model(loaded_gaussiangb_plain_sklearn, 'Loaded GaussianGB', test_v2[3:5])

Loaded GaussianGB prediction: [0 0]

The prediction is for the following data:
     age                        job  marital          education  default housing loan   contact day_of_week day_of_week_cos  campaign previous     poutcome  cons_conf_idx  euribor3m nr_employed bought
3 34.000              self_employed   single       basic_school       no      no   no  cellular           1           0.309         1        0  nonexistent        -42.000      4.153    5195.800      0
4 36.000  administration_management  married  university.degree  unknown      no   no  cellular           0           1.000         4        0  nonexistent        -36.100      4.963    5228.100      0


In [71]:
# Making another set of predictions
loaded_gaussiangb_plain_sklearn.predict(test_v2.loc[31:35, test_v2.columns[0:-1]])

array([1, 1, 0, 0, 0])

In [72]:
test_v2.loc[31:35, :].bought.values

[0, 1, 0, 0, 0]
Categories (2, int64): [0, 1]

## Loading the saved XBGBoost model and making a prediction

In [41]:
loaded_random_search_xgb = load('random_search_xgb_plain_sklearn.joblib')

get_prediction_loaded_model(loaded_random_search_xgb, 'Loaded XGBoost', test_v2[3:5])

Loaded XGBoost prediction: [0 0]

The prediction is for the following data:
     age                        job  marital          education  default housing loan   contact day_of_week day_of_week_cos  campaign previous     poutcome  cons_conf_idx  euribor3m nr_employed bought
3 34.000              self_employed   single       basic_school       no      no   no  cellular           1           0.309         1        0  nonexistent        -42.000      4.153    5195.800      0
4 36.000  administration_management  married  university.degree  unknown      no   no  cellular           0           1.000         4        0  nonexistent        -36.100      4.963    5228.100      0


In [73]:
# Making another set of predictions
loaded_random_search_xgb.predict(test_v2.loc[31:35, test_v2.columns[0:-1]])

array([1, 0, 0, 0, 0])